In [3]:
from clrs._src.algorithms.graphs import bfs

In [4]:
import numpy as np
from torch_geometric.data import Data
import networkx as nx

In [5]:
class CLRSData(Data):
    """A data object for CLRS data."""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [11]:
import numpy as np
import networkx as nx
from clrs._src.algorithms.graphs import bfs
import matplotlib.pyplot as plt
import torch
class graph_generator:
    def __init__(self, n, p, directed=False, type='erdos_renyi'):
        self.n = n # number of nodes
        self.p = p # probability of edge creation
        self.s = np.random.randint(0, n) # source node
        if type == 'erdos_renyi':
            self.graph = nx.erdos_renyi_graph(n, p, directed=directed)
        self.adj = nx.to_numpy_array(self.graph)
        self.edges_indexes = self.get_edges_indexes(self.adj)

        self.pi, probes = bfs(self.adj, self.s)
        self.pi_h = probes['hint']['node']['pi_h']['data']
        self.reach_h = probes['hint']['node']['reach_h']['data']

        self.pi = self.get_edges(self.adj, self.pi)
        self.pi_h = np.array([self.get_edges(self.adj, x) for x in self.pi_h])
        self.pos = np.arange(0, n)/n
        self.length = (self.pi_h).shape[0] # length of the sequence of hints

        dict = {'edge_index': self.edges_indexes, 'pos': self.pos, 'length': self.length, 's': self.s, 'pi': self.pi, 'reach_h': self.reach_h, 'pi_h': self.pi_h}
        dict = {k: self.to_torch(v) for k,v in dict.items()}
        dict['hints'] = np.array(['reach_h', 'pi_h'])
        dict['inputs'] = np.array(['pos', 's'])
        dict['outputs'] = np.array(['pi'])
        self.tensor = CLRSData(**dict)

    def get_edges_indexes(self, A):
        edge_indexes = [[],[]]
        """Create a 2x1 list of lists to store the indexes of the edges in the adjacency matrix."""
        for i in range(len(A)):
            for j in range(len(A)):
                if A[i][j] == 1:
                    edge_indexes[0].append(i)
                    edge_indexes[1].append(j)
        return edge_indexes
    
    def get_edges(self, A, pi):
        """Returns a binary list of edges in the graph. If the edge was accessed by the algorithm, it is marked as 1, otherwise it is 0."""
        edge_indexes = self.get_edges_indexes(A)
        edges = np.zeros(len(edge_indexes[0]))
        for i in range(len(pi)):
            if pi[i] != -1:
                for j in range(len(edge_indexes[0])):
                    if edge_indexes[1][j] == i and edge_indexes[0][j] == pi[i]:
                        edges[j] = 1
        return edges
    
    def draw(self):
        nx.draw(self.graph, with_labels=True)
        plt.show()
    
    def to_torch(self, value):
        if isinstance(value, np.ndarray):
            return torch.from_numpy(value)
        elif isinstance(value, torch.Tensor):
            return value
        else:
            return torch.tensor(value)
        
    def save(self, path):
        torch.save(self.tensor, path)

In [12]:
# test de la clase
n =10
p = 0.3
g = graph_generator(n, p)

In [13]:
from concurrent.futures import ThreadPoolExecutor
class graph_generator_batch:
    def __init__(self, n, p, path, directed=False, max_workers=2):
        self.n = np.array(n)
        self.p = np.array(p)
        self.directed = directed
        self.path = path
        self.max_workers = max_workers
    
    def generate(self, num_graphs):
        print('Generating ' + str(num_graphs) + ' graphs')
        for i in range(num_graphs):
            self.generate_graph(i)

    def generate_graph(self, i):
        g = graph_generator(np.random.randint(self.n[0], self.n[1]), np.random.uniform(self.p[0], self.p[1]), directed=self.directed)
        g.save(self.path + '/graph' + str(i) + '.pt')

In [14]:
import os
n = [10, 20]
p = [0.3, 0.5]
path = os.getcwd() + '/data'

In [15]:
num_graphs = 10
g = graph_generator_batch(n, p, path)
g.generate(num_graphs)

Generating 10 graphs


In [16]:
# access the first graph
graph = torch.load(path + '/graph0.pt')
print(graph)

CLRSData(edge_index=[2, 66], pos=[15], length=4, s=2, pi=[66], reach_h=[4, 15], pi_h=[4, 66], hints=[2], inputs=[2], outputs=[1])


In [ ]:
# create an encoder class
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(Encoder, self).__init__()
        
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.conv2(x, edge_index)
        return x